# Hack [nairaland.com](https://www.nairaland.com/)

Please the accompanying `README.md` before proceeding.

Several demos of this project in action are provided below. Just change the cell type to `code` to run the examples.

In [1]:
import glob
import hack

import pandas as pd

In [ ]:
# hack.export_user_comments_to_html(username="VargasVee", max_page=3)
hack.export_user_comments_to_excel(username="VargasVee", max_page=3)

In [ ]:
hack.export_topics_to_html(section='romance', start=0, stop=4)
hack.export_topics_to_excel(section='romance', start=0, stop=4)

In [2]:
e = glob.glob('{}/*.xlsx'.format(hack.BASE_DIR))
e

['C:\\Users\\Chidimmo\\Desktop\\hack-nairaland\\comments_raumdeuter_3_pages.xlsx',
 'C:\\Users\\Chidimmo\\Desktop\\hack-nairaland\\~$comments_raumdeuter_3_pages.xlsx']

In [10]:
df = pd.read_excel(e[0])
df.style.set_properties(**{'text-align': 'left', 'background-color': 'green', 'color' : 'white', 'font-size' : '125%'})


,SECTION,TOPIC,USER_COMMENT,QUOTED_USER
0,"European Football (EPL, UEFA, La Liga)",Official Chelsea Fan Thread: 2018 FA cup winner,Seriously,Lana1: Son heard you!
1,nan,nan,nan,nan
2,"European Football (EPL, UEFA, La Liga)**2",Official Chelsea Fan Thread: 2018 FA cup winner,1 goal before 65mins and it's on,nan
3,"European Football (EPL, UEFA, La Liga)**4",Official Chelsea Fan Thread: 2018 FA cup winner,"You keep screaming pass the ball, to who? If you've been following Chelsea since the start of the season, you'll find out how many passes and crosses in the box no one got to","RuuDie: I swear if Hazard can just learn to pass or cross the ball after one or two touches, Chelsea strikers will probably score more... he will hold ball and begin to caress it like Mike and his sex doll! Very annoying mofo!!"
4,nan,nan,nan,nan
5,"European Football (EPL, UEFA, La Liga)**6",Official Chelsea Fan Thread: 2018 FA cup winner,"Whoever said pass to Morata isn't watching, he's confused as hell. Big ups to Kepa. All we need is one early goal in the second half. COYB!",nan
6,"European Football (EPL, UEFA, La Liga)**8",Official Chelsea Fan Thread: 2018 FA cup winner,Lord I missed this thread. Feels like forever Nice to see raumdeuter still here,nan
7,Romance,I'm Afraid,More power to your waist bro..,nan
8,Education,"N10k To 10 Students Each(Phase 1, 2, 3,4, 5, 6, 7 8(9&10) Grand finale Ended..)",Good one,nan
9,Education**14,"N10k To 10 Students Each(Phase 1, 2, 3,4, 5, 6, 7 8(9&10) Grand finale Ended..)",nan,nan


import textwrap

for page in list(UserCommentHistory("preccy69").scrap_comments_for_page_range(start=0, stop=1)):
    for section, topic_plus_comment in page.items():
        print("\n\n", "*"*40, section, "*"*40)
        print(topic_plus_comment.topic.upper()) # for differentiation only

        parsed_comment = topic_plus_comment.parsed_comment # a namedtuple instance
        print(parsed_comment.focus_user_comment)

        quotes = parsed_comment.quotes_ordered_dict
        for username, comment in quotes.items():
            print(" "*8)
            print(textwrap.indent(username, "    "))
            print(textwrap.indent(comment, "    "))
        print("_"*100)
    print("\n\n")

import textwrap

for page in TopicCollector(section='politics').scrap_topics_for_range_of_pages(start=0, stop=1):
    for topic in list(page):
        print(topic.poster)
        print(textwrap.indent(topic.title, "    "))
        print(textwrap.indent(topic.url, "    "))
        
        print(textwrap.indent(topic.comments, "    "), " comments") # str is necessary because of the way textwrap works
        print(textwrap.indent(topic.views, "    "), " views")
        print(textwrap.indent(topic.last_commenter, "    "), " commented last")
        print(textwrap.indent(topic.other_meta), "    ")
        print()

In [ ]:
def export_post_to_docx_format(post_url):
    """Export post to word"""

    post = PostCollector(post_url)
    comments_and_commenters = post.view_comments_commenters()

    document = docx.Document()
    document.add_paragraph(post_url)

    for each in comments_and_commenters:
        commenter = each[0]
        comment_block = each[1]

        quotes = single_string_from_dictionary(comment_block[0])
        user_comments = comment_block[1]

        document.add_paragraph().add_run(commenter).bold = True
        document.add_paragraph().add_run('[{}]'.format(quotes)).italic = True
        document.add_paragraph(user_comments)
        document.add_paragraph('*'*50)

    fname = "{}.docx".format(post.post_title)
    document.save(os.path.join(BASE_DIR, fname))

In [ ]:
def title_word_count(section='romance', max_page=10):
    """Group words that occur in titles of a section

    Parameters
    -----------
    str
        Section
    int
        Maximum number of section pages to scrap

    Returns
    --------
    word_count : list
        Dictionary of each word and its count
    list
        A list of all words sorted in descending order of frequency
    """
    word_list = []
    section_object = TopicCollector(section=section).titles_links_metadata(end=max_page)
    section_data = list(section_object)
    split_string = r"[\; \, \n \.+\- \( \) - \/ : \? \[ \] \ — –]"

    exclude = ['to', 'a', 'as', 'the', 'you', 'in', 'is', 'i', 'with', 'of', 'an', 'and', 'my',
               'your', 'for', 'on', 'what', 'her', 'this', 'that', 'these', 'those', 'me']

    for each in section_data:
        title = each[3]
        words_in_title = re.split(split_string, title)
        word_list.extend(words_in_title)

    word_list = [x.lower() for x in word_list]
    word_list = filter(lambda x: x not in exclude, word_list)
    word_count = Counter(list(word_list))
    return word_count, sort_dictionary_by_value(word_count)